In [88]:
import pandas as pd
import numpy as np 
import datetime
import os
from treelib import Node, Tree
import pickle

In [76]:
data_test = pd.read_csv('E://天池竞赛/杭州地铁乘客流量/Metro_testA/testA_record_2019-01-28.csv')
data_submit = pd.read_csv('E://天池竞赛/杭州地铁乘客流量/Metro_testA/testA_submit_2019-01-29.csv')

In [91]:
in_test_pd = pd.DataFrame(columns=range(82))
out_test_pd = pd.DataFrame(columns=range(82))
start_time = data_test.time[1][:10] + ' 00:00'
tm_rang = pd.date_range(start_time,periods=6*24,freq='10T')
for i in range(len(tm_rang)):

    in_test_data = {}
    out_test_data = {}
    start_index  = datetime.datetime.fromtimestamp(tm_rang[i].value/1000000000 - 28800).strftime('%Y-%m-%d %H:%M:%S')
    if i != len(tm_rang)-1:
        end_index = datetime.datetime.fromtimestamp(tm_rang[i+1].value/1000000000  - 28800).strftime('%Y-%m-%d %H:%M:%S')
        sub_time_data = data_test[(data_test['time']>=start_index) & (data_test['time']<=end_index)]
    else:
        print(tm_rang[i])
        sub_time_data = data_test[(data_test['time']>=start_index)]
    for stationID, group in sub_time_data.groupby('stationID'):
        total_num = len(group)
        in_num = group[group.status==1].describe().status['count']
        out_num = total_num - in_num
        in_test_data[stationID] = in_num
        out_test_data[stationID] = out_num
    in_test_data['81'] = tm_rang[i]
    out_test_data['81'] = tm_rang[i]
    in_test_pd = in_test_pd.append(in_test_data, ignore_index=True)
    out_test_pd = out_test_pd.append(out_test_data, ignore_index=True)

d:\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexes\range.py:453: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  return self._int64index.union(other)
d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects


2019-01-28 23:50:00


In [98]:

pickle.dump(in_test_pd, open('in_test1_pd.pkl', 'wb'))
pickle.dump(out_test_pd, open('out_test1_pd.pkl', 'wb'))

In [96]:
in_num_pd = pd.DataFrame(columns=range(82))
out_num_pd = pd.DataFrame(columns=range(82))
rootdir = "E://天池竞赛/杭州地铁乘客流量/Metro_train/"
list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
for i in range(0,len(list)):
    path = os.path.join(rootdir,list[i])
    if os.path.isfile(path):
        print(path)
        data_train_sample = pd.read_csv(path)
        start_time = data_train_sample.time[1][:10] + ' 00:00'
        tm_rang = pd.date_range(start_time,periods=6*24,freq='10T')
        for i in range(len(tm_rang)):
            
            in_data = {}
            out_data = {}
            start_index  = datetime.datetime.fromtimestamp(tm_rang[i].value/1000000000 - 28800).strftime('%Y-%m-%d %H:%M:%S')
            if i != len(tm_rang)-1:
                end_index = datetime.datetime.fromtimestamp(tm_rang[i+1].value/1000000000  - 28800).strftime('%Y-%m-%d %H:%M:%S')
                sub_time_data = data_train_sample[(data_train_sample['time']>=start_index) & (data_train_sample['time']<=end_index)]
            else:
                print(tm_rang[i])
                sub_time_data = data_train_sample[(data_train_sample['time']>=start_index)]
            for stationID, group in sub_time_data.groupby('stationID'):
                total_num = len(group)
                in_num = group[group.status==1].describe().status['count']
                out_num = total_num - in_num
                in_data[stationID] = in_num
                out_data[stationID] = out_num
            in_data['81'] = tm_rang[i]
            out_data['81'] = tm_rang[i]
            in_num_pd = in_num_pd.append(in_data, ignore_index=True)
            out_num_pd = out_num_pd.append(out_data, ignore_index=True)

E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-01.csv


d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects


2019-01-01 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-02.csv
2019-01-02 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-03.csv
2019-01-03 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-04.csv
2019-01-04 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-05.csv
2019-01-05 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-06.csv
2019-01-06 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-07.csv
2019-01-07 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-08.csv
2019-01-08 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-09.csv
2019-01-09 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-10.csv
2019-01-10 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-11.csv
2019-01-11 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-12.csv
2019-01-12 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-13.csv
2019-01-13 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-14.csv
2019-01-14 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01

In [97]:
import pickle
pickle.dump(in_num_pd, open('in_num1_pd.pkl', 'wb'))
pickle.dump(out_num_pd, open('out_num1_pd.pkl', 'wb'))

In [89]:
in_num_pd = pickle.load(open('in_num1_pd.pkl', 'rb'))
out_num_pd = pickle.load(open('out_num1_pd.pkl', 'rb'))

In [ ]:
def predict_values(values, future_periods = 90):
    prophet = Prophet()
    train_df = pd.DataFrame({
        'ds': data,
        'y' : y
    })
    train_df['y'] = np.log(train_df['y'])
    prophet.fit(train_df)
    future = tsmodel.make_future_dataframe(periods=future_periods, freq='10T')
    forecast = prophet.predict(future)
    mae = np.sum(np.absolute(forecast['yhat'].head(len(train_df))-train_df['y']))/len(train_df)
    print('MAE:' + mae)
    y_predict = forecast['yhat'].tail(future_periods).values
    return y_predict

In [ ]:
mae = np.sum(np.absolute(forecast['yhat'].head(len(train_df))-train_df['y']))/len(train_df)

In [ ]:
def BFS(node0):
    #queue本质上是堆栈，用来存放需要进行遍历的数据
    #order里面存放的是具体的访问路径
    queue,order = [],[]
    #首先将初始遍历的节点放到queue中，表示将要从这个点开始遍历
    # 由于是广度优先，也就是先访问初始节点的所有的子节点，所以可以
    queue.append(node0)
    order.append(node0)
    while queue:
        #queue.pop(0)意味着是队列的方式出元素，就是先进先出，而下面的for循环将节点v的所有子节点
        #放到queue中，所以queue.pop(0)就实现了每次访问都是先将元素的子节点访问完毕，而不是优先叶子节点
        v = queue.pop(0)
        for w in self.sequense[v]:
            if w not in order:
                # 这里可以直接order.append(w) 因为广度优先就是先访问节点的所有下级子节点，所以可以
                # 将self.sequense[v]的值直接全部先给到order
                order.append(w)
                queue.append(w)
    return order

In [60]:
class stationTree():
    def __init__(self,start_station, maxDeep, mapArray):
        self.stationTree = Tree()
        self.total = []
        self.total.append(start_station)
        self.start_station = start_station
        self.maxDeep = maxDeep
        self.mapArray = mapArray
        
    def creat_tree_by_BFS(self):
        queue = []
        queue.append(self.start_station)
        self.stationTree.create_node(self.start_station, self.start_station)
        while queue and self.stationTree.depth()<self.maxDeep:
            v = queue.pop(0)
            for i in range(len(self.mapArray)):
                if self.mapArray[v][i] == 1 and i not in self.total:
                    queue.append(i)
                    self.total.append(i)
                    self.stationTree.create_node(i, i, parent=v)

In [61]:
roadMap = pd.read_csv('E://天池竞赛/杭州地铁乘客流量/Metro_roadMap.csv', index_col=0)
mapArray = roadMap.values
start_id = 4
s_tree = stationTree(start_id, 25, mapArray)
s_tree.creat_tree_by_BFS()
print(s_tree.stationTree.size())
s_tree.stationTree.show()

81
4
├── 3
│   └── 2
│       └── 1
│           └── 0
└── 5
    ├── 6
    │   └── 7
    │       └── 8
    │           └── 9
    │               ├── 10
    │               │   ├── 11
    │               │   │   └── 12
    │               │   │       └── 13
    │               │   │           └── 14
    │               │   ├── 50
    │               │   │   └── 49
    │               │   └── 52
    │               │       └── 53
    │               │           └── 54
    │               │               └── 55
    │               │                   └── 56
    │               │                       └── 57
    │               │                           └── 58
    │               │                               └── 59
    │               │                                   └── 60
    │               │                                       └── 61
    │               │                                           └── 62
    │               │                                               └── 63


In [65]:
import statsmodels.formula.api as smf

In [101]:
submit_data = pd.read_csv("E://天池竞赛/杭州地铁乘客流量/Metro_testA/testA_submit_2019-01-29.csv")

In [102]:
submit_data

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,NaN,NaN
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,NaN,NaN
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,NaN,NaN
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,NaN,NaN
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,NaN,NaN
5,0,2019-01-29 00:50:00,2019-01-29 01:00:00,NaN,NaN
6,0,2019-01-29 01:00:00,2019-01-29 01:10:00,NaN,NaN
7,0,2019-01-29 01:10:00,2019-01-29 01:20:00,NaN,NaN
8,0,2019-01-29 01:20:00,2019-01-29 01:30:00,NaN,NaN
9,0,2019-01-29 01:30:00,2019-01-29 01:40:00,NaN,NaN


In [91]:
in_pred = pickle.load(open('in_pred_pd.pkl','rb'))
out_pred = pickle.load(open('out_pred_pd.pkl','rb'))

In [125]:
pred_29_in = in_pred.tail(144)
pred_29_out = out_pred.tail(144)

In [149]:
pred_29_in_arr = [i for item in pred_29_in.values.astype(np.int).T for i in item]
pred_29_out_arr = [i for item in pred_29_out.values.astype(np.int).T for i in item]

In [152]:
pred_29_in_arr = pred_29_in.values.astype(np.int).T.reshape(1,-1)
pred_29_out_arr = pred_29_out.values.astype(np.int).T.reshape(1,-1)

In [140]:
len(pred_29_in_arr)

1

In [148]:
pred_29_in.values.astype(np.int).T[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   1,   1,   3,   5,   9,  16,
        27,  45,  73, 113, 169, 242, 331, 431, 536, 633, 713, 765, 783,
       769, 726, 662, 586, 506, 429, 359, 298, 246, 205, 172, 146, 126,
       111, 100,  92,  87,  84,  84,  85,  89,  94, 101, 110, 120, 133,
       147, 161, 176, 190, 204, 214, 222, 226, 226, 222, 215, 204, 191,
       178, 163, 149, 136, 124, 114, 105,  98,  92,  88,  86,  84,  84,
        86,  89,  93,  98, 105, 112, 120, 129, 137, 144, 149, 152, 152,
       148, 140, 129, 115,  99,  83,  67,  52,  40,  29,  21,  15,  10,
         7,   4,   3,   2,   1,   0,   0,   0,   0,   0,   0,   0,   0,
         0])

In [153]:
submit_data['inNums'] = pred_29_in_arr[0]
submit_data['outNums'] = pred_29_out_arr[0]

In [134]:

a = np.array([[2,3], [2,3],[1,1]])
print(a)
print(a.reshape(1, -1))

[[2 3]
 [2 3]
 [1 1]]
[[2 3 2 3 1 1]]


In [154]:
submit_data.to_csv('result.csv')